Hello 🙌, welcome to my notebook. In this notebook we will try to exploring house sales data and also develop and evaluate model to predict house sales price. Feel free if you have any question or suggestion! Thank you!

* Before i start, there is several piece of code that i copy from this beatufiul Kernel:

https://www.kaggle.com/burhanykiyakoglu/predicting-house-prices

https://www.kaggle.com/kabure/predicting-house-prices-xgb-rf-bagging-reg-pipe

Kindly, check and upvote their Kernel. Thank you!

# Intro & Feature Defenition

Intro: This dataset contains house sale prices for King County, which includes Seattle. It includes homes sold between May 2014 and May 2015. It's a great dataset for evaluating simple regression models.

- id - Unique ID for each home sold
- date - Date of the home sale
- price - Price of each home sold
- bedrooms - Number of bedrooms
- bathrooms - Number of bathrooms, where .5 accounts for a room with a toilet but no shower
- sqft_living - Square footage of the apartments interior living space
- sqft_lot - Square footage of the land space
- floors - Number of floors
- waterfront - A dummy variable for whether the apartment was overlooking the waterfront or not
- view - An index from 0 to 4 of how good the view of the property was
- condition - An index from 1 to 5 on the condition of the apartment,
- grade - An index from 1 to 13, where 1-3 falls short of building construction and design, 7 has an average level of construction and design, and 11-13 have a high quality level of construction and design.
- sqft_above - The square footage of the interior housing space that is above ground level
- sqft_basement - The square footage of the interior housing space that is below ground level
- yr_built - The year the house was initially built
- yr_renovated - The year of the house’s last renovation
- zipcode - What zipcode area the house is in
- lat - Lattitude
- long - Longitude
- sqft_living15 - The square footage of interior housing living space for the nearest 15 neighbors
- sqft_lot15 - The square footage of the land lots of the nearest 15 neighbors

# Data Importing & Preview

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

data1 = pd.read_csv('../input/housesalesprediction/kc_house_data.csv')

In [ ]:
data1.head()

In [ ]:
data1.shape

In [ ]:
data1.info()

In [ ]:
data1['date'] = pd.to_datetime(data1['date'].str.strip(), format='%Y/%m/%d')

In [ ]:
data1['Year'] = data1['date'].dt.year # getting year
data1['Month'] = data1['date'].dt.month # getting month
data1['Day'] = data1['date'].dt.day # getting day

In [ ]:
data1.head()

In [ ]:
data1.describe()

In [ ]:
'''Missing Value Chart'''
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
data1.isnull().mean(axis=0).plot.barh()
plt.title("Ratio of missing values per columns")

In [ ]:
'''Checking Duplicate'''

print('Dupplicate entries: {}'.format(data1.duplicated().sum()))
# data1.drop_duplicates(inplace = True)

In [ ]:
#label encoding
from sklearn.preprocessing import LabelEncoder

var = ['id','zipcode']

for i in var:
    labelencoder = LabelEncoder()
    data1[i] = labelencoder.fit_transform(data1[i])

In [ ]:
data1['zipcode'].nunique()

- We have 21613 rows, 21 faetures
- And for Date i convert to datetime format, and generating it's Year, Month, and Day in seperate column
- Our data is clear from missing values and duplicate
- For id and zipcode, i will convert it to simple numerical label using LabelEncoder

# Number of Sold House

In [ ]:
import plotly.express as px
import plotly.offline as py 
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go 
import plotly.tools as tools
import warnings
from collections import Counter 
from plotly.subplots import make_subplots

custom_aggregation = {}
custom_aggregation["id"] = "count"
data2 = data1.groupby("Year").agg(custom_aggregation)
data2['Year'] = data2.index

fig = make_subplots(rows=1, 
                    cols=2, 
                    specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=('Year',
                                    'Month'))


labels = data2['Year'].tolist()
values = data2['id'].tolist()

fig.add_trace(go.Pie(
                    labels=labels,
                    values=values, 
                    name=''),
                    1,1)

#-------------------------------------------

custom_aggregation = {}
custom_aggregation["id"] = "count"
data2 = data1.groupby("Month").agg(custom_aggregation)
data2['Month'] = data2.index

labels = data2['Month'].tolist()
values = data2['id'].tolist()


fig.add_trace(go.Pie(
                    labels=labels,
                    values=values, 
                    name=''),
                    1,2)

fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig['layout'].update(height=400, 
                     width=800, 
                     title='Number of Sold House in:',
                     showlegend=False)
fig.show()

- In 2014, number of house for sold is 67.7%, greater than in 2015 which is 32.3%
- Throughout the month of the year, most houses are sold in May

# Mean Price Distribution

In [ ]:
fig = make_subplots(rows=1, 
                    cols=1)

custom_aggregation = {}
custom_aggregation["price"] = "mean"
data2 = data1.groupby("zipcode").agg(custom_aggregation)
data2.columns = ["Mean Price"]
data2['Region'] = data2.index
data2['Region'] = data2['Region'].astype('category')
data2 = data2.sort_values(by = 'Mean Price', ascending = False)

x = data2['Region'].tolist()
y = data2['Mean Price'].tolist()


fig.add_trace(go.Scatter(x=x, 
                         y=y,
                         name=' ',
                         mode='markers'), 1, 1)

fig['layout'].update(height=500, 
                     width=900, 
                     title='Mean Price Distribution by Region',
                    xaxis_title="Region",
                    yaxis_title="Mean Price",
                    legend_title="Price")
fig.show()

In [ ]:
data1['price'].describe()

In [ ]:
q1 = data1['price'].describe()[4]
q3 = data1['price'].describe()[6]
q1, q3

In [ ]:
def conditions(a):
    if (a['price'] <= q1):
        return 'Low'
    if (a['price'] >= q3):
        return 'High'
    else:
        return 'Medium'

In [ ]:
data1['Cat.'] = data1.apply(conditions, axis=1)

In [ ]:
import plotly.express as px

# Price

custom_aggregation = {}
custom_aggregation["price"] = "mean"
custom_aggregation["Cat."] = lambda x: x.mode().iloc[0]
data2 = data1.groupby("zipcode").agg(custom_aggregation)
data2.columns = ["Mean Price",'Category']
data2['Region'] = data2.index

fig = px.scatter(data2, 
              x="Region", 
              y="Mean Price", 
              color="Category")

fig['layout'].update(height=500, 
                     width=900, 
                     title='Mean Price Distribution by Region',
                    xaxis_title="Region",
                    yaxis_title="Mean Price",
                    legend_title="Price")

fig.show()

- I try to plot mean price of house in each region
- But i think it will be more usefull if we can categorized wether the region categorized as High, Medium or Low house price
- To do that i make a function to categorized its region
- If the house price sold greater than Q3 it will categorized as High
- If the house price sold lower than Q1 it will categorized as Lower
- Else will return as Medium

# Feature Distribution

In [ ]:
# Bedrooms

custom_aggregation = {}
custom_aggregation["bedrooms"] = 'mean'
custom_aggregation["Cat."] = lambda x: x.mode().iloc[0]
data2 = data1.groupby("zipcode").agg(custom_aggregation)
data2.columns = ["Bedrooms",'Category']
data2['Region'] = data2.index

fig = px.scatter(data2, 
              x="Region", 
              y="Bedrooms", 
              color="Category")

fig['layout'].update(height=500, 
                     width=900, 
                     title='Bedrooms Distribution by Region',
                    xaxis_title="Region",
                    yaxis_title="Bedrooms",
                    legend_title="Price")

fig.show()

In [ ]:
# Bathrooms

custom_aggregation = {}
custom_aggregation["bathrooms"] = 'mean'
custom_aggregation["Cat."] = lambda x: x.mode().iloc[0]
data2 = data1.groupby("zipcode").agg(custom_aggregation)
data2.columns = ["Bathrooms",'Category']
data2['Region'] = data2.index

fig = px.scatter(data2, 
              x="Region", 
              y="Bathrooms", 
              color="Category")

fig['layout'].update(height=500, 
                     width=900, 
                     title='Bathrooms Distribution by Region',
                    xaxis_title="Region",
                    yaxis_title="Bathrooms",
                    legend_title="Price")

fig.show()

In [ ]:
# Living Area

custom_aggregation = {}
custom_aggregation["sqft_living"] = 'mean'
custom_aggregation["Cat."] = lambda x: x.mode().iloc[0]
data2 = data1.groupby("zipcode").agg(custom_aggregation)
data2.columns = ["Living Area",'Category']
data2['Region'] = data2.index

fig = px.scatter(data2, 
              x="Region", 
              y="Living Area", 
              color="Category")

fig['layout'].update(height=500, 
                     width=900, 
                     title='Living Area Distribution by Region',
                    xaxis_title="Region",
                    yaxis_title="Living Area",
                    legend_title="Price")

fig.show()

In [ ]:
# Land Space

custom_aggregation = {}
custom_aggregation["sqft_lot"] = 'mean'
custom_aggregation["Cat."] = lambda x: x.mode().iloc[0]
data2 = data1.groupby("zipcode").agg(custom_aggregation)
data2.columns = ["Land Space",'Category']
data2['Region'] = data2.index

fig = px.scatter(data2, 
              x="Region", 
              y="Land Space", 
              color="Category")

fig['layout'].update(height=500, 
                     width=900, 
                     title='Land Space Distribution by Region',
                    xaxis_title="Region",
                    yaxis_title="Land Space",
                    legend_title="Price")

fig.show()

In [ ]:
# Floors

custom_aggregation = {}
custom_aggregation["floors"] = 'mean'
custom_aggregation["Cat."] = lambda x: x.mode().iloc[0]
data2 = data1.groupby("zipcode").agg(custom_aggregation)
data2.columns = ["Floors",'Category']
data2['Region'] = data2.index

fig = px.scatter(data2, 
              x="Region", 
              y="Floors", 
              color="Category")

fig['layout'].update(height=500, 
                     width=900, 
                     title='Floors Distribution by Region',
                    xaxis_title="Region",
                    yaxis_title="Floors",
                    legend_title="Price")

fig.show()

In [ ]:
# Waterfront

custom_aggregation = {}
custom_aggregation["waterfront"] = 'mean'
custom_aggregation["Cat."] = lambda x: x.mode().iloc[0]
data2 = data1.groupby("zipcode").agg(custom_aggregation)
data2.columns = ["Waterfront",'Category']
data2['Region'] = data2.index

fig = px.scatter(data2, 
              x="Region", 
              y="Waterfront", 
              color="Category")

fig['layout'].update(height=500, 
                     width=900, 
                     title='Waterfront Distribution by Region',
                    xaxis_title="Region",
                    yaxis_title="Waterfront",
                    legend_title="Price")

fig.show()

In [ ]:
# View

custom_aggregation = {}
custom_aggregation["view"] = 'mean'
custom_aggregation["Cat."] = lambda x: x.mode().iloc[0]
data2 = data1.groupby("zipcode").agg(custom_aggregation)
data2.columns = ["View",'Category']
data2['Region'] = data2.index

fig = px.scatter(data2, 
              x="Region", 
              y="View", 
              color="Category")

fig['layout'].update(height=500, 
                     width=900, 
                     title='View Distribution by Region',
                    xaxis_title="Region",
                    yaxis_title="View",
                    legend_title="Price")

fig.show()

In [ ]:
# Condition

custom_aggregation = {}
custom_aggregation["condition"] = 'mean'
custom_aggregation["Cat."] = lambda x: x.mode().iloc[0]
data2 = data1.groupby("zipcode").agg(custom_aggregation)
data2.columns = ["Condition",'Category']
data2['Region'] = data2.index

fig = px.scatter(data2, 
              x="Region", 
              y="Condition", 
              color="Category")

fig['layout'].update(height=500, 
                     width=900, 
                     title='Condition Distribution by Region',
                    xaxis_title="Region",
                    yaxis_title="Condition",
                    legend_title="Price")

fig.show()

In [ ]:
# Grade

custom_aggregation = {}
custom_aggregation["grade"] = 'mean'
custom_aggregation["Cat."] = lambda x: x.mode().iloc[0]
data2 = data1.groupby("zipcode").agg(custom_aggregation)
data2.columns = ["Grade",'Category']
data2['Region'] = data2.index

fig = px.scatter(data2, 
              x="Region", 
              y="Grade", 
              color="Category")

fig['layout'].update(height=500, 
                     width=900, 
                     title='Grade Distribution by Region',
                    xaxis_title="Region",
                    yaxis_title="Grade",
                    legend_title="Price")

fig.show()

In [ ]:
# SQFT Above

custom_aggregation = {}
custom_aggregation["sqft_above"] = 'mean'
custom_aggregation["Cat."] = lambda x: x.mode().iloc[0]
data2 = data1.groupby("zipcode").agg(custom_aggregation)
data2.columns = ["SQFT Above",'Category']
data2['Region'] = data2.index

fig = px.scatter(data2, 
              x="Region", 
              y="SQFT Above", 
              color="Category")

fig['layout'].update(height=500, 
                     width=900, 
                     title='SQFT Above Distribution by Region',
                    xaxis_title="Region",
                    yaxis_title="SQFT Above",
                    legend_title="Price")

fig.show()

In [ ]:
# SQFT Basement

custom_aggregation = {}
custom_aggregation["sqft_basement"] = 'mean'
custom_aggregation["Cat."] = lambda x: x.mode().iloc[0]
data2 = data1.groupby("zipcode").agg(custom_aggregation)
data2.columns = ["SQFT Basement",'Category']
data2['Region'] = data2.index

fig = px.scatter(data2, 
              x="Region", 
              y="SQFT Basement", 
              color="Category")

fig['layout'].update(height=500, 
                     width=900, 
                     title='SQFT Basement Distribution by Region',
                    xaxis_title="Region",
                    yaxis_title="SQFT Basement",
                    legend_title="Price")

fig.show()

In [ ]:
# Renovated

custom_aggregation = {}
custom_aggregation["yr_renovated"] = 'mean'
custom_aggregation["Cat."] = lambda x: x.mode().iloc[0]
data2 = data1.groupby("zipcode").agg(custom_aggregation)
data2.columns = ["Renovated",'Category']
data2['Region'] = data2.index

fig = px.scatter(data2, 
              x="Region", 
              y="Renovated", 
              color="Category")

fig['layout'].update(height=500, 
                     width=900, 
                     title='Renovated Distribution by Region',
                    xaxis_title="Region",
                    yaxis_title="Renovated",
                    legend_title="Price")

fig.show()

In [ ]:
# Built

custom_aggregation = {}
custom_aggregation["yr_built"] = 'mean'
custom_aggregation["Cat."] = lambda x: x.mode().iloc[0]
data2 = data1.groupby("zipcode").agg(custom_aggregation)
data2.columns = ["Built",'Category']
data2['Region'] = data2.index

fig = px.scatter(data2, 
              x="Region", 
              y="Built", 
              color="Category")

fig['layout'].update(height=500, 
                     width=900, 
                     title='Built Distribution by Region',
                    xaxis_title="Region",
                    yaxis_title="Built",
                    legend_title="Price")

fig.show()

In [ ]:
# SQFT Living 15

custom_aggregation = {}
custom_aggregation["sqft_living15"] = 'mean'
custom_aggregation["Cat."] = lambda x: x.mode().iloc[0]
data2 = data1.groupby("zipcode").agg(custom_aggregation)
data2.columns = ["SQFT Living 15",'Category']
data2['Region'] = data2.index

fig = px.scatter(data2, 
              x="Region", 
              y="SQFT Living 15", 
              color="Category")

fig['layout'].update(height=500, 
                     width=900, 
                     title='SQFT Living 15 Distribution by Region',
                    xaxis_title="Region",
                    yaxis_title="SQFT Living 15",
                    legend_title="Price")

fig.show()

In [ ]:
# SQFT Lot 15

custom_aggregation = {}
custom_aggregation["sqft_lot15"] = 'mean'
custom_aggregation["Cat."] = lambda x: x.mode().iloc[0]
data2 = data1.groupby("zipcode").agg(custom_aggregation)
data2.columns = ["SQFT Lot 15",'Category']
data2['Region'] = data2.index

fig = px.scatter(data2, 
              x="Region", 
              y="SQFT Lot 15", 
              color="Category")

fig['layout'].update(height=500, 
                     width=900, 
                     title='SQFT Lot 15 Distribution by Region',
                    xaxis_title="Region",
                    yaxis_title="SQFT Lot 15",
                    legend_title="Price")

fig.show()

# Price Category Comparison on Each Feature

In [ ]:
custom_aggregation = {}
custom_aggregation["price"] = 'mean'
custom_aggregation["bedrooms"] = 'mean'
custom_aggregation["bathrooms"] = 'mean'
custom_aggregation["sqft_living"] = 'mean'
custom_aggregation["sqft_lot"] = 'mean'
custom_aggregation["floors"] = 'mean'
custom_aggregation["waterfront"] = 'mean'
custom_aggregation["view"] = 'mean'
custom_aggregation["condition"] = 'mean'
custom_aggregation["grade"] = 'mean'
custom_aggregation["sqft_above"] = 'mean'
custom_aggregation["sqft_basement"] = 'mean'
custom_aggregation["yr_built"] = 'mean'
custom_aggregation["yr_renovated"] = 'mean'
custom_aggregation["zipcode"] = lambda x: x.mode().iloc[0]
custom_aggregation["sqft_living15"] = 'mean'
custom_aggregation["sqft_lot15"] = 'mean'

data2 = data1.groupby("Cat.").agg(custom_aggregation)
data2['Feature'] = data2.index

d_ = go.Bar(
    x = data2['Feature'].value_counts().index.sort_values(),
    y = data2["price"],
    name='Price')

d__ = go.Bar(
    x = data2['Feature'].value_counts().index.sort_values(),
    y = data2["bedrooms"],
    name='Bedrooms')

d___ = go.Bar(
    x = data2['Feature'].value_counts().index.sort_values(),
    y = data2["bathrooms"],
    name='Bathrooms')


data = [d_, d__, d___]

fig = tools.make_subplots(rows=1, 
                          cols=3)

fig.append_trace(d_, 1, 1)
fig.append_trace(d__, 1, 2)
fig.append_trace(d___, 1, 3)

fig['layout'].update(height=400, 
                     width=800, 
                     title='Price Category Comparison on Each Feature',
                    xaxis_title=" ",
                    yaxis_title=" ",
                    legend_title="Feature")

py.iplot(fig, filename='combined-savings')



In [ ]:
d_ = go.Bar(
    x = data2['Feature'].value_counts().index.sort_values(),
    y = data2["sqft_living"],
    name='SQFT Living')

d__ = go.Bar(
    x = data2['Feature'].value_counts().index.sort_values(),
    y = data2["sqft_lot"],
    name='SQFT Lot')

d___ = go.Bar(
    x = data2['Feature'].value_counts().index.sort_values(),
    y = data2["floors"],
    name='Floors')


data = [d_, d__, d___]

fig = tools.make_subplots(rows=1, 
                          cols=3)

fig.append_trace(d_, 1, 1)
fig.append_trace(d__, 1, 2)
fig.append_trace(d___, 1, 3)

fig['layout'].update(height=400, 
                     width=800, 
                     title='Price Category Comparison on Each Feature',
                    xaxis_title=" ",
                    yaxis_title=" ",
                    legend_title="Feature")

py.iplot(fig, filename='combined-savings')

In [ ]:
d_ = go.Bar(
    x = data2['Feature'].value_counts().index.sort_values(),
    y = data2["waterfront"],
    name='Waterfront')

d__ = go.Bar(
    x = data2['Feature'].value_counts().index.sort_values(),
    y = data2["view"],
    name='View')

d___ = go.Bar(
    x = data2['Feature'].value_counts().index.sort_values(),
    y = data2["condition"],
    name='Condition')


data = [d_, d__, d___]

fig = tools.make_subplots(rows=1, 
                          cols=3)

fig.append_trace(d_, 1, 1)
fig.append_trace(d__, 1, 2)
fig.append_trace(d___, 1, 3)

fig['layout'].update(height=400, 
                     width=800, 
                     title='Price Category Comparison on Each Feature',
                    xaxis_title=" ",
                    yaxis_title=" ",
                    legend_title="Feature")

py.iplot(fig, filename='combined-savings')

In [ ]:
d_ = go.Bar(
    x = data2['Feature'].value_counts().index.sort_values(),
    y = data2["grade"],
    name='Grade')

d__ = go.Bar(
    x = data2['Feature'].value_counts().index.sort_values(),
    y = data2["sqft_above"],
    name='SQFT Above')

d___ = go.Bar(
    x = data2['Feature'].value_counts().index.sort_values(),
    y = data2["sqft_basement"],
    name='SQFT Basement')


data = [d_, d__, d___]

fig = tools.make_subplots(rows=1, 
                          cols=3)

fig.append_trace(d_, 1, 1)
fig.append_trace(d__, 1, 2)
fig.append_trace(d___, 1, 3)

fig['layout'].update(height=400, 
                     width=800, 
                     title='Price Category Comparison on Each Feature',
                    xaxis_title=" ",
                    yaxis_title=" ",
                    legend_title="Feature")

py.iplot(fig, filename='combined-savings')

In [ ]:
d_ = go.Bar(
    x = data2['Feature'].value_counts().index.sort_values(),
    y = data2["yr_built"],
    name='Built')

d__ = go.Bar(
    x = data2['Feature'].value_counts().index.sort_values(),
    y = data2["yr_renovated"],
    name='Renovated')

d___ = go.Bar(
    x = data2['Feature'].value_counts().index.sort_values(),
    y = data2["sqft_living15"],
    name='SQFT Living 15')


d____ = go.Bar(
    x = data2['Feature'].value_counts().index.sort_values(),
    y = data2["sqft_lot15"],
    name='SQFT Lot 15')


data = [d_, d__, d___,d____]

fig = tools.make_subplots(rows=1, 
                          cols=4)

fig.append_trace(d_, 1, 1)
fig.append_trace(d__, 1, 2)
fig.append_trace(d___, 1, 3)
fig.append_trace(d____, 1, 4)

fig['layout'].update(height=400, 
                     width=900, 
                     title='Price Category Comparison on Each Feature',
                    xaxis_title=" ",
                    yaxis_title=" ",
                    legend_title="Feature")

py.iplot(fig, filename='combined-savings')

- I want to find out differences from each category of house (Low,Medium, High) in each features.
- The higher difference in each house category, it will show that these features will greatly affect the price of the house
- And feature that affect price based on significance difference on house category are:

    1. Renovated, 
    2. SQFT Living 15
    3. SQFT Lot 15
    4. SQFT Basement
    5. SQFT Above
    6. View
    7. Waterfront
    8. SQFT Living
    9. SQFT Lot
    
    
- Assume that you really want home with nice view but in affordable price, i think you should go to city number 65 and 35 which have Medium Price. Or city number 12 and 68 which have low price!

# Pre-Processing

In [ ]:
data1['Year'].min(), data1['Year'].max()

In [ ]:
data1.corr()['price'].sort_values(ascending=False)

In [ ]:
features = data1.corr()['price'].drop(['price','lat','long','Year']).sort_values(ascending=False)[:15].index.tolist()
features

In [ ]:
fig = make_subplots(rows=1, 
                    cols=1)
cor = data1.corr()
cor_ = cor.index
cor__ = cor.values

fig.add_trace(go.Heatmap(
                    x=cor_,
                    y=cor_,
                    z=cor__,
                    name='Correlation',
                    showscale=False,
                    xgap=0.7,
                    ygap=0.7), 1, 1)


fig['layout'].update(height=600, 
                     width=600, 
                     title='Heat Map',
                     xaxis_title=" ",
                     yaxis_title=" ")
fig.show()

In [ ]:
from sklearn.model_selection import train_test_split

train_data,test_data = train_test_split(data1,train_size = 0.7,random_state=3)

In [ ]:
train_data.shape, test_data.shape

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import Lasso, Ridge, ElasticNet, RANSACRegressor, SGDRegressor, HuberRegressor, BayesianRidge
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from xgboost import XGBRegressor, plot_importance
from sklearn.svm import SVR, SVC, LinearSVC
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.decomposition import KernelPCA, PCA
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import learning_curve, validation_curve, GridSearchCV
from sklearn.base import clone
from sklearn.metrics import mean_squared_error as MSE

pipelines = []

seed = 2

pipelines.append(("Scaled_Ridge", 
                  Pipeline([("Scaler", StandardScaler()),
                            ("Ridge", Ridge(random_state=seed, tol=10 ))
                           ]
                          )
                 )
                )


pipelines.append(("Scaled_Lasso", 
                 Pipeline([("Scaler", StandardScaler()), 
                           ("Lasso", Lasso(random_state=seed, tol=1))
                          ]
                         )
                 )
                )

pipelines.append(("Scaled_Elastic", 
                 Pipeline([("Scaler", StandardScaler()), 
                           ("Lasso", ElasticNet(random_state=seed))
                          ]
                         )
                 )
                )

pipelines.append(("Scaled_SVR",
                 Pipeline([("Scaler", StandardScaler()),
                           ("SVR",  SVR(kernel='linear', C=1e2, degree=5))
                          ]
                         )
                 )
                )

pipelines.append(("Scaled_RF_reg",
                 Pipeline([("Scaler", StandardScaler()),
                           ("RF", RandomForestRegressor(random_state=seed))
                          ]
                         )
                 )
                )

pipelines.append(("Scaled_ET_reg",
                 Pipeline([("Scaler", StandardScaler()),
                           ("ET", ExtraTreesRegressor(random_state=seed))
                          ]
                         )
                 )
                )

pipelines.append(("Scaled_BR_reg",
                 Pipeline([("Scaler", StandardScaler()),
                           ("BR", BaggingRegressor(random_state=seed))
                          ]
                         )
                 )
                ) 

pipelines.append(("Scaled_Hub-Reg",
                 Pipeline([("Scaler", StandardScaler()),
                           ("Hub-Reg", HuberRegressor())
                          ]
                         )
                 )
                )

pipelines.append(("Scaled_BayRidge",
                 Pipeline([("Scaler", StandardScaler()),
                        ("BR", BayesianRidge())
                          ]
                         )
                 )
                ) 

pipelines.append(("Scaled_XGB_reg",
                 Pipeline([("Scaler", StandardScaler()),
                          ("XGBR", XGBRegressor(seed=seed))
                          ]
                         )
                 )
                ) 

pipelines.append(("Scaled_DT_reg",
                 Pipeline([("Scaler", StandardScaler()),
                           ("DT_reg", DecisionTreeRegressor())
                          ]
                         )
                 )
                ) 

pipelines.append(("Scaled_KNN_reg",
                 Pipeline([("Scaler", StandardScaler()),
                           ("KNN_reg", KNeighborsRegressor())
                          ]
                         )
                 )
                )

pipelines.append(("Scaled_ADA-Reg",
                  Pipeline([("Scaler", StandardScaler()),
                            ("ADA-reg", AdaBoostRegressor())
                           ]
                          )
                 )
                ) 

pipelines.append(("Scaled_Gboost-Reg",
                 Pipeline([("Scaler", StandardScaler()),
                           ("GBoost-Reg", GradientBoostingRegressor())
                          ]
                         )
                 )
                )

pipelines.append(("Scaled_RFR_PCA",
                 Pipeline([("Scaler", StandardScaler()),
                           ("PCA", PCA(n_components=3)),
                           ("XGB", RandomForestRegressor())
                          ]
                         )
                 )
                )

pipelines.append(("Scaled_XGBR_PCA",
                 Pipeline([("Scaler", StandardScaler()),
                           ("PCA", PCA(n_components=3)),
                           ("XGB", XGBRegressor())
                          ]
                         )
                 )
                )

#'neg_mean_absolute_error', 'neg_mean_squared_error','r2'
scoring = 'r2'
n_folds = 7
results, names  = [], [] 

for name, model  in pipelines:
    kfold = KFold(n_splits=n_folds, random_state=None)
    cv_results = cross_val_score(model, train_data[features], train_data['price'], cv= kfold,scoring=scoring, n_jobs=-1)    
    names.append(name)
    results.append(cv_results)    
    msg = "%s: %f (+/- %f)" % (name, cv_results.mean(),  cv_results.std())
    print(msg)
    
# boxplot algorithm comparison
fig = plt.figure(figsize=(18,6))
fig.suptitle('Algorithm Comparison by R2')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.xticks(rotation=90)
plt.show()

- For the feature, i select only feature which have positive correlation with price
- Before modelling i will review several algorithm based on R2 values
- I decided to go with Random Forest, Extra Tree, XGB and Debugging

# Modelling

In [ ]:
evaluation = pd.DataFrame({'Model': [],
                           'Details':[],
                           'Root Mean Squared Error (RMSE)':[],
                           'R-squared (training)':[],
                           'Adjusted R-squared (training)':[],
                           'R-squared (test)':[],
                           'Adjusted R-squared (test)':[],
                           '5-Fold Cross Validation':[]})

In [ ]:
def adjustedR2(r2,n,k):
    return r2-(k-1)/(n-k)*(1-r2)

In [ ]:
'''RANDOM FOREST REGRESION'''
#with multiple selected_feature
import numpy as np
from sklearn import metrics

rf = RandomForestRegressor(random_state=seed)
rf.fit(train_data[features],train_data['price'])

pred1 = rf.predict(test_data[features])
rmsecm1 = float(format(np.sqrt(metrics.mean_squared_error(test_data['price'],pred1)),'.3f'))
rtrcm1 = float(format(rf.score(train_data[features],train_data['price']),'.3f'))
artrcm1 = float(format(adjustedR2(rf.score(train_data[features],train_data['price']),train_data.shape[0],len(features)),'.3f'))
rtecm1 = float(format(rf.score(test_data[features],test_data['price']),'.3f'))
artecm1 = float(format(adjustedR2(rf.score(test_data[features],test_data['price']),test_data.shape[0],len(features)),'.3f'))
cv1 = float(format(cross_val_score(rf,data1[features],data1['price'],cv=5).mean(),'.3f'))

r = evaluation.shape[0]
evaluation.loc[r] = ['Random Forest','selected features',rmsecm1,rtrcm1,artrcm1,rtecm1,artecm1,cv1]
evaluation.sort_values(by = '5-Fold Cross Validation', ascending=False)
evaluation

In [ ]:
'''EXTRA TREE REGRESION'''
#with multiple selected_feature

et = ExtraTreesRegressor(random_state=seed)
et.fit(train_data[features],train_data['price'])

pred2 = et.predict(test_data[features])
rmsecm2 = float(format(np.sqrt(metrics.mean_squared_error(test_data['price'],pred2)),'.3f'))
rtrcm2 = float(format(et.score(train_data[features],train_data['price']),'.3f'))
artrcm2 = float(format(adjustedR2(et.score(train_data[features],train_data['price']),train_data.shape[0],len(features)),'.3f'))
rtecm2 = float(format(et.score(test_data[features],test_data['price']),'.3f'))
artecm2 = float(format(adjustedR2(et.score(test_data[features],test_data['price']),test_data.shape[0],len(features)),'.3f'))
cv2 = float(format(cross_val_score(et,data1[features],data1['price'],cv=5).mean(),'.3f'))

r = evaluation.shape[0]
evaluation.loc[r] = ['Extra Tree','selected features',rmsecm2,rtrcm2,artrcm2,rtecm2,artecm2,cv2]
evaluation.sort_values(by = '5-Fold Cross Validation', ascending=False)
evaluation

In [ ]:
'''XGB REGRESION'''
#with multiple selected_feature

xgb = XGBRegressor(random_state=seed)
xgb.fit(train_data[features],train_data['price'])

pred3 = xgb.predict(test_data[features])
rmsecm3 = float(format(np.sqrt(metrics.mean_squared_error(test_data['price'],pred3)),'.3f'))
rtrcm3 = float(format(xgb.score(train_data[features],train_data['price']),'.3f'))
artrcm3 = float(format(adjustedR2(xgb.score(train_data[features],train_data['price']),train_data.shape[0],len(features)),'.3f'))
rtecm3 = float(format(xgb.score(test_data[features],test_data['price']),'.3f'))
artecm3 = float(format(adjustedR2(xgb.score(test_data[features],test_data['price']),test_data.shape[0],len(features)),'.3f'))
cv3 = float(format(cross_val_score(xgb,data1[features],data1['price'],cv=5).mean(),'.3f'))

r = evaluation.shape[0]
evaluation.loc[r] = ['XGB Regressor','selected features',rmsecm3,rtrcm3,artrcm3,rtecm3,artecm3,cv3]
evaluation.sort_values(by = '5-Fold Cross Validation', ascending=False)
evaluation

In [ ]:
'''BAGGING REGRESION'''
#with multiple selected_feature

bg = BaggingRegressor(random_state=seed)
bg.fit(train_data[features],train_data['price'])

pred4 = bg.predict(test_data[features])
rmsecm4 = float(format(np.sqrt(metrics.mean_squared_error(test_data['price'],pred4)),'.3f'))
rtrcm4 = float(format(bg.score(train_data[features],train_data['price']),'.3f'))
artrcm4 = float(format(adjustedR2(bg.score(train_data[features],train_data['price']),train_data.shape[0],len(features)),'.3f'))
rtecm4 = float(format(bg.score(test_data[features],test_data['price']),'.3f'))
artecm4 = float(format(adjustedR2(bg.score(test_data[features],test_data['price']),test_data.shape[0],len(features)),'.3f'))
cv4 = float(format(cross_val_score(bg,data1[features],data1['price'],cv=5).mean(),'.3f'))

r = evaluation.shape[0]
evaluation.loc[r] = ['Bagging','selected features',rmsecm4,rtrcm4,artrcm4,rtecm4,artecm4,cv4]
evaluation.sort_values(by = '5-Fold Cross Validation', ascending=False)
evaluation

In [ ]:
# DISPLAY MODELLING
import seaborn as sns

def model_performance_sc_plot(predictions, labels, title):
    min_val = max(max(predictions), max(labels))
    max_val = min(min(predictions), min(labels))
    performance_df = pd.DataFrame({"Label":labels})
    performance_df["Prediction"] = predictions
    sns.jointplot(y="Label", x="Prediction", data=performance_df, kind="reg", height=7)
    plt.plot([min_val, max_val], [min_val, max_val], 'm--')
    plt.title(title, fontsize=9)
    plt.show()

In [ ]:
# Random Forest

model_performance_sc_plot(pred1, test_data['price'], 'Validation')

In [ ]:
# Extra Tree

model_performance_sc_plot(pred2, test_data['price'], 'Validation')

In [ ]:
# XGB

model_performance_sc_plot(pred3, test_data['price'], 'Validation')

In [ ]:
# Bagging

model_performance_sc_plot(pred4, test_data['price'], 'Validation')

# Prediction

In [ ]:
data1['Random Forest'] = rf.predict(data1[features])
data1['Extra Tree'] = et.predict(data1[features])
data1['XGB'] = xgb.predict(data1[features])
data1['Bagging'] = bg.predict(data1[features])

In [ ]:
data_ = data1[['id','price','Random Forest','Extra Tree','XGB','Bagging']]
data_.sort_values(by='id', ascending=True).head()

Finish. Don't forget to upvote! Thank you!:)